<b><h2>What we are going to do in this notebook : -</h2></b>

**First we are going to discuss about LB and then how to divide the test set into Public LB and Private LB and then do Adversarial Validation on both test sets with the trainsets .**

If you find any mistake I have made here then Please report that in the coments.

<font color='red'>**If you like this notebook then Plese Upvote :)**

In [1]:
import os
import numpy as np 
import pandas as pd 

pd.set_option("max_columns", 500)
pd.set_option("max_rows", 500)

In [2]:
train = pd.read_csv("../input/tabular-playground-series-sep-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-sep-2022/test.csv")
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

train = train.drop(["num_sold"], axis=1)
print(train.shape, test.shape)

(70128, 5) (17520, 5)


In [3]:
train.head(5)

,row_id,date,country,store,product
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques


There are two versions of Leaderboards in Kaggle Competitions :
1. <b><u>Public LB : </u></b> This is the Leaderboard which is available to us during any competition, it consists of a portion of a test set, (In this competition this is 25% of the total test data)

2. <b><u>Private LB : </u></b> This is the Leaderboard which is available to us after the competition ends, it decides who wins the Medals and final points.(In this competition this is 75% of the total test data)

Most of the time(specially in tougher competitions) the distributions between Public and Private LBs are set to be different, in those times despite getting a higher position during the competition sometimes results in getting a very lower position at the end of the competition. It depends on the submissions you choose as final 2. (1)If the Private LB is more like Public LB then you may choose the best 2 LB solutions as your final 2 but if it's not the case,(2) If the Private LB is completely different from Public LB then it's better to choose the submissions with BEST local CVs.

**How will we detect if both of the LBs are alike or not?**

1) In `Non-Code` competitions it's relatively easier since we are already given the sample_submission & test file, we have to split the test set in a way that it resembles the public-private set ratio(given at the top of the LB webpage).

2) Then we just submit the sample_submission file to Kaggle(unchanged) and note down the score we get as `base_score`.

3) After we have chosen the splits, we need to replace the private submission part as a higher number as 999 and keep the public part same(as the sample_submission file) and then submit to Kaggle, and then compare the score we get to the`base_score`. If both scores are same then comgrats you have successfully identified the splits!!!

**Why does it work?**

Since the LB will only show us the result of Public Set it won't mind the 999 values we have put in the private submission part as long as we have ONLY modified the private part.

In [4]:
print(test.shape)
test.head(5)

(17520, 5)


,row_id,date,country,store,product
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques


**Public-Private Split for this competition**

* In this competition we are given a test set of 17520 elements (for 2021, 365 days for each day we have to predict for 6 countries and 8 samples per country)

* Since the public set the 25% of the whole data, it means it will contain the elements of 3 months, it's most likely that these 3 months will be consecutive.

* I first submitted the sample_submission to get the `base score` : 
![](https://i.postimg.cc/Vkpw6MnK/Screenshot-2022-09-06-at-00-13-21-Tabular-Playground-Series-Sep-2022-Kaggle.png)

* Then I kept first 3 months(JAN-MARCH) same(`num_sold` : 100) and changed the others to 999, and it worked! I got the same score as the sample_submission!
![](https://i.postimg.cc/2657KqXf/Screenshot-2022-09-06-at-00-16-37-Tabular-Playground-Series-Sep-2022-Kaggle.png)

* To be sure I double checked it with keeping March-May same(`num_sold` : 100) and putting others to 999 and the score increased!
![](https://i.postimg.cc/SxNLsYkY/Screenshot-2022-09-06-at-00-18-35-Tabular-Playground-Series-Sep-2022-Kaggle.png)


***Which clearly means that the PUBLIC LB IS FROM JAN-MARCH & the PRIVATE LB IS APRIL-DEC***

In [5]:
public_test = test[test.date<="2021-03-31"]
private_test = test[test.date>="2021-04-01"]

print(public_test.shape, private_test.shape)
print(public_test.tail(5))
print(private_test.head(5))

(4320, 5) (13200, 5)
      row_id       date country       store                           product
4315   74443 2021-03-31   Spain  KaggleMart  Kaggle for Kids: One Smart Goose
4316   74444 2021-03-31   Spain  KaggleRama        Kaggle Advanced Techniques
4317   74445 2021-03-31   Spain  KaggleRama            Kaggle Getting Started
4318   74446 2021-03-31   Spain  KaggleRama                Kaggle Recipe Book
4319   74447 2021-03-31   Spain  KaggleRama  Kaggle for Kids: One Smart Goose
      row_id       date  country       store                           product
4320   74448 2021-04-01  Belgium  KaggleMart        Kaggle Advanced Techniques
4321   74449 2021-04-01  Belgium  KaggleMart            Kaggle Getting Started
4322   74450 2021-04-01  Belgium  KaggleMart                Kaggle Recipe Book
4323   74451 2021-04-01  Belgium  KaggleMart  Kaggle for Kids: One Smart Goose
4324   74452 2021-04-01  Belgium  KaggleRama        Kaggle Advanced Techniques


ADVERSARIAL VALIDATION

* After we have splitted the LB into Public and Private, we need to check if they are different from each other or not.

* One way to do it that we can train a classifier to predict the Public and Private sets, (1)If the classifier manages to predict them(AUC close to 1.0) then they are different from each other, (2) Otherwise if the classifier gets score of(AUC close to 0.5-0.6)the sets are so intertwined that the classifier can't also pick them apart! 

* We can do this for all of them (TrainVSPublic, TrainVSPrivate and PublicVSPrivate) we do this one by one below. 

* We remove the target variable(num_sold) from train and then combine both sets and then also consider Train set as 0 and Public Set as 1.

**TRAIN VS PUBLIC-TEST**

In [6]:
# TRAIN : 0
# PUBLIC TEST : 1
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tt = pd.concat([train.drop(["row_id"], axis=1), public_test.drop(["row_id"], axis=1)])
tt['target'] = [0]*train.shape[0]+[1]*public_test.shape[0]
tt = tt.sample(frac=1.0).reset_index(drop=True)

print(tt.shape)
print(tt.head(5))
print(tt['target'].value_counts())

for col in tt.columns:
    if col!="target":
        print(col)
        le = LabelEncoder()
        tt.loc[:, col] = le.fit_transform(tt[col])
        
        
# FIT MODEL
x_tr, x_val, y_tr, y_val = train_test_split(tt.drop(["target"], axis=1), tt["target"],
                                           test_size=0.2)
print(x_tr.shape, x_val.shape, y_tr.shape, y_val.shape)
clf = xgb.XGBClassifier(n_estimators=50,
                       eval_metric="auc")
clf.fit(x_tr, y_tr,
       eval_set=[(x_tr, y_tr), (x_val, y_val)],
       verbose=10)

(74448, 5)
        date  country       store                     product  target
0 2021-02-26  Germany  KaggleMart  Kaggle Advanced Techniques       1
1 2018-04-07  Germany  KaggleMart  Kaggle Advanced Techniques       0
2 2017-05-03   Poland  KaggleMart  Kaggle Advanced Techniques       0
3 2017-01-04    Italy  KaggleRama  Kaggle Advanced Techniques       0
4 2018-11-22   Poland  KaggleMart      Kaggle Getting Started       0
0    70128
1     4320
Name: target, dtype: int64
date
country
store
product
(59558, 4) (14890, 4) (59558,) (14890,)
[0]	validation_0-auc:1.00000	validation_1-auc:1.00000
[10]	validation_0-auc:1.00000	validation_1-auc:1.00000
[20]	validation_0-auc:1.00000	validation_1-auc:1.00000
[30]	validation_0-auc:1.00000	validation_1-auc:1.00000
[40]	validation_0-auc:1.00000	validation_1-auc:1.00000
[49]	validation_0-auc:1.00000	validation_1-auc:1.00000


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

* We, can see that the score is very high!!! AUC=1.0 but it is all due to the `date` column. Since the date in Train is from `2017-01-01` to `2020-12-31`,  and the `date` in Public Set is from `2021-01-01` to `2021-03-31`. 
* If we remove it we will see that the datasets are no different from each other, but we must not remove this feature(as we may do in other cases to make these two sets same) since it will make the data useless. But it is still good to know that the sets are same! 

**TRAIN VS PUBLIC-TEST(WITHOUT `DATE`)**

In [7]:
# TRAIN : 0
# PUBLIC TEST : 1
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tt = pd.concat([train.drop(["row_id", "date"], axis=1), public_test.drop(["row_id", "date"], axis=1)])
tt['target'] = [0]*train.shape[0]+[1]*public_test.shape[0]
tt = tt.sample(frac=1.0).reset_index(drop=True)

print(tt.shape)
print(tt.head(5))
print(tt['target'].value_counts())

for col in tt.columns:
    if col!="target":
        print(col)
        le = LabelEncoder()
        tt.loc[:, col] = le.fit_transform(tt[col])
        
        
# FIT MODEL
x_tr, x_val, y_tr, y_val = train_test_split(tt.drop(["target"], axis=1), tt["target"],
                                           test_size=0.2)
print(x_tr.shape, x_val.shape, y_tr.shape, y_val.shape)
clf = xgb.XGBClassifier(n_estimators=50,
                       eval_metric="auc")
clf.fit(x_tr, y_tr,
       eval_set=[(x_tr, y_tr), (x_val, y_val)],
       verbose=10)

(74448, 4)
   country       store                     product  target
0    Italy  KaggleRama  Kaggle Advanced Techniques       0
1   France  KaggleMart      Kaggle Getting Started       0
2   Poland  KaggleRama      Kaggle Getting Started       0
3  Germany  KaggleMart  Kaggle Advanced Techniques       0
4    Italy  KaggleRama      Kaggle Getting Started       0
0    70128
1     4320
Name: target, dtype: int64
country
store
product
(59558, 3) (14890, 3) (59558,) (14890,)
[0]	validation_0-auc:0.50000	validation_1-auc:0.50000
[10]	validation_0-auc:0.51258	validation_1-auc:0.44923
[20]	validation_0-auc:0.51441	validation_1-auc:0.44182
[30]	validation_0-auc:0.51443	validation_1-auc:0.44173
[40]	validation_0-auc:0.51443	validation_1-auc:0.44173
[49]	validation_0-auc:0.51443	validation_1-auc:0.44173


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

**THE OTHER AVs are shown below...**

**PUBLIC-TEST VS PRIVATE-TEST(WITHOUT `DATE`)**

In [8]:
# PUBLIC TEST : 0
# PRIVATE TEST : 1
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tt = pd.concat([public_test.drop(["row_id", "date"], axis=1), private_test.drop(["row_id", "date"], axis=1)])
tt['target'] = [0]*public_test.shape[0]+[1]*private_test.shape[0]
tt = tt.sample(frac=1.0).reset_index(drop=True)

print(tt.shape)
print(tt.head(5))
print(tt['target'].value_counts())

for col in tt.columns:
    if col!="target":
        print(col)
        le = LabelEncoder()
        tt.loc[:, col] = le.fit_transform(tt[col])
        
        
# FIT MODEL
x_tr, x_val, y_tr, y_val = train_test_split(tt.drop(["target"], axis=1), tt["target"],
                                           test_size=0.2)
print(x_tr.shape, x_val.shape, y_tr.shape, y_val.shape)
clf = xgb.XGBClassifier(n_estimators=50,
                       eval_metric="auc")
clf.fit(x_tr, y_tr,
       eval_set=[(x_tr, y_tr), (x_val, y_val)],
       verbose=10)

(17520, 4)
  country       store                           product  target
0   Spain  KaggleMart  Kaggle for Kids: One Smart Goose       1
1  Poland  KaggleRama                Kaggle Recipe Book       1
2   Italy  KaggleMart        Kaggle Advanced Techniques       1
3   Spain  KaggleRama        Kaggle Advanced Techniques       0
4  France  KaggleRama            Kaggle Getting Started       1
1    13200
0     4320
Name: target, dtype: int64
country
store
product
(14016, 3) (3504, 3) (14016,) (3504,)
[0]	validation_0-auc:0.50000	validation_1-auc:0.50000
[10]	validation_0-auc:0.51658	validation_1-auc:0.43454
[20]	validation_0-auc:0.51681	validation_1-auc:0.43362
[30]	validation_0-auc:0.51681	validation_1-auc:0.43361
[40]	validation_0-auc:0.51681	validation_1-auc:0.43361
[49]	validation_0-auc:0.51681	validation_1-auc:0.43361


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

**TRAIN VS PRIVATE TEST(WITHOUT `DATE`)**

In [9]:
# TRAIN : 0
# PRIVATE TEST : 1
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tt = pd.concat([train.drop(["row_id", "date"], axis=1), private_test.drop(["row_id", "date"], axis=1)])
tt['target'] = [0]*train.shape[0]+[1]*private_test.shape[0]
tt = tt.sample(frac=1.0).reset_index(drop=True)

print(tt.shape)
print(tt.head(5))
print(tt['target'].value_counts())

for col in tt.columns:
    if col!="target":
        print(col)
        le = LabelEncoder()
        tt.loc[:, col] = le.fit_transform(tt[col])
        
        
# FIT MODEL
x_tr, x_val, y_tr, y_val = train_test_split(tt.drop(["target"], axis=1), tt["target"],
                                           test_size=0.2)
print(x_tr.shape, x_val.shape, y_tr.shape, y_val.shape)
clf = xgb.XGBClassifier(n_estimators=50,
                       eval_metric="auc")
clf.fit(x_tr, y_tr,
       eval_set=[(x_tr, y_tr), (x_val, y_val)],
       verbose=10)

(83328, 4)
   country       store                           product  target
0    Spain  KaggleMart            Kaggle Getting Started       1
1  Germany  KaggleRama            Kaggle Getting Started       0
2  Belgium  KaggleRama  Kaggle for Kids: One Smart Goose       1
3    Spain  KaggleMart  Kaggle for Kids: One Smart Goose       0
4    Spain  KaggleMart                Kaggle Recipe Book       1
0    70128
1    13200
Name: target, dtype: int64
country
store
product
(66662, 3) (16666, 3) (66662,) (16666,)
[0]	validation_0-auc:0.50000	validation_1-auc:0.50000
[10]	validation_0-auc:0.50909	validation_1-auc:0.46373
[20]	validation_0-auc:0.50922	validation_1-auc:0.46318
[30]	validation_0-auc:0.50922	validation_1-auc:0.46317
[40]	validation_0-auc:0.50922	validation_1-auc:0.46317
[49]	validation_0-auc:0.50922	validation_1-auc:0.46317


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

**Colclusion :**

* It clearly shows that there is no difference in distribution between `TRAINvsPUBLIC-TEST`, `TRAINvsPRIVATE-TEST`, `PRIVATE-TESTvsPUBLIC-TEST` BUT WITHOUT DATE!

* So, we need to make a validation that completely resembles the Private Set.